In [ ]:
import pandas
import pickle
import seaborn

In [ ]:
with open("measure-rtdp.pkl", "rb") as f:
    results = pickle.load(f)
d = results["data"]
list(d.columns)
d["fullmodel"] = d.apply(lambda x: f"{x.protocol}-{x.model}-trunc{x.trunc}", axis=1)
d.columns

In [ ]:
log = []
for _, row in d.query('algo == "rtdp"').iterrows():
    for entry in row.log:
        params = dict()
        for p in ["fullmodel", "attacker", "hyperparams"]:
            params[p] = row[p]
        log.append(dict(algo="rtdp") | params | entry)

        ref_row = d.query(
            f'fullmodel == "{row.fullmodel}" and attacker  == "{row.attacker}" and algo == "aft20"'
        )
        assert ref_row.shape[0] == 1
        ref_row = ref_row.iloc[0]
        ref_entry = dict(
            step=entry["step"],
            start_value=ref_row.start_value,
            start_progress=ref_row.start_progress,
            n_states=ref_row.pimc_n_states,
        )
        log.append(dict(algo="aft20") | params | ref_entry)

log = pandas.DataFrame(log)
log

In [ ]:
seaborn.relplot(
    data=log,
    kind="line",
    x="step",
    y="start_value",
    col="attacker",
    hue="algo",
    row="fullmodel",
)

In [ ]:
seaborn.relplot(
    data=log.assign(rpp=lambda x: x.start_value / x.start_progress),
    kind="line",
    x="step",
    y="rpp",
    col="attacker",
    hue="algo",
    row="fullmodel",
)

In [ ]:
seaborn.relplot(
    # data = log.query('algo == "rtdp"'),
    data=log,
    kind="line",
    x="step",
    y="n_states",
    col="attacker",
    hue="algo",
    row="fullmodel",
)